# GradientBoostingRegressor

2017-05-22

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validation, GridSearchCV
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date,time

/home/godcedric/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/godcedric/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### 录入数据

In [2]:
travel_time_train_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/加工过的数据集/2.0/以最近值填充的/travel_time_train_data.csv')
volume_train_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/加工过的数据集/2.0/以最近值填充的/volume_train_data.csv')
travel_time_test_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/加工过的数据集/2.0/以最近值填充的/test_travel_time_data.csv')
volume_test_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/加工过的数据集/2.0/以最近值填充的/test_volume_data.csv')

### 准备数据

In [3]:
# 分割训练集和验证集
test_date = ['2016-10-11', '2016-10-12', '2016-10-13', '2016-10-14', '2016-10-15', '2016-10-16', '2016-10-17']
def isin(x):
    if x in test_date:
        return 1
    else:
        return 0
# 平均时间
travel_time_train_data['is_test'] = travel_time_train_data['date'].map(isin)
travel_time_train = travel_time_train_data[travel_time_train_data['is_test'] == 0]
travel_time_val = travel_time_train_data[travel_time_train_data['is_test'] == 1]
del travel_time_train['is_test']
del travel_time_val['is_test']
# 流量
volume_train_data['is_test'] = volume_train_data['date'].map(isin)
volume_train = volume_train_data[volume_train_data['is_test'] == 0]
volume_val = volume_train_data[volume_train_data['is_test'] == 1]
del volume_train['is_test']
del volume_val['is_test']

In [4]:
# 分离标签和特征
Y_time = travel_time_train.avg_travel_time
X_time = travel_time_train.drop(['avg_travel_time'], axis=1)
val_Y_time = travel_time_val.avg_travel_time
val_X_time = travel_time_val.drop(['avg_travel_time'], axis=1)


Y_volume = volume_train.volume
X_volume = volume_train.drop(['volume'], axis=1)
val_Y_volume = volume_val.volume
val_X_volume = volume_val.drop(['volume'], axis=1)

In [5]:
# 平均时间
# 选择特征
X_time = X_time.drop(['intersection_id', 'tollgate_id', 'time_window', 'start_time', 'date', 'time'], axis=1)
X_time = X_time.drop(['wind_direction', 'wind_speed', 'precipitation', 'SSD'], axis=1)
X_time = X_time.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
X_time = X_time.drop(['datemap'], axis=1)

val_X_time = val_X_time.drop(['intersection_id', 'tollgate_id', 'time_window', 'start_time', 'date', 'time'], axis=1)
val_X_time = val_X_time.drop(['wind_direction', 'wind_speed', 'precipitation', 'SSD'], axis=1)
val_X_time = val_X_time.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
val_X_time = val_X_time.drop(['datemap'], axis=1)

# 测试集同步
travel_time_submission = travel_time_test_data[['intersection_id', 'tollgate_id', 'time_window']]
travel_time_test_data = travel_time_test_data.drop(['intersection_id', 'tollgate_id', 'time_window', 'start_time', 'date', 'time'], axis=1)
travel_time_test_data = travel_time_test_data.drop(['wind_direction', 'wind_speed', 'precipitation', 'SSD'], axis=1)
travel_time_test_data = travel_time_test_data.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
travel_time_test_data = travel_time_test_data.drop(['datemap'], axis=1)

In [6]:
# 流量
# 选择特征
X_volume = X_volume.drop(['tollgate_id', 'direction', 'time_window', 'start_time', 'date', 'time'], axis=1)
X_volume = X_volume.drop(['wind_direction', 'wind_speed', 'precipitation', 'SSD'], axis=1)
X_volume = X_volume.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
X_volume = X_volume.drop(['datemap'], axis=1)

val_X_volume = val_X_volume.drop(['tollgate_id', 'direction', 'time_window', 'start_time', 'date', 'time'], axis=1)
val_X_volume = val_X_volume.drop(['wind_direction', 'wind_speed', 'precipitation', 'SSD'], axis=1)
val_X_volume = val_X_volume.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
val_X_volume = val_X_volume.drop(['datemap'], axis=1)

# 测试集同步
volume_submission = volume_test_data[['tollgate_id', 'time_window', 'direction']]
volume_test_data = volume_test_data.drop(['tollgate_id', 'direction', 'time_window', 'start_time', 'date', 'time'], axis=1)
volume_test_data = volume_test_data.drop(['wind_direction', 'wind_speed', 'precipitation', 'SSD'], axis=1)
volume_test_data = volume_test_data.drop(['last_40min', 'last_60min', 'last_80min', 'last_100min', 'last_120min'], axis=1)
volume_test_data = volume_test_data.drop(['datemap'], axis=1)

In [7]:
# 字符串特征转换数值
X_time['route'] = pd.factorize(X_time['route'])[0]
val_X_time['route'] = pd.factorize(val_X_time['route'])[0]
travel_time_test_data['route'] = pd.factorize(travel_time_test_data['route'])[0]
X_volume['pair'] = pd.factorize(X_volume['pair'])[0]
val_X_volume['pair'] = pd.factorize(val_X_volume['pair'])[0]
volume_test_data['pair'] = pd.factorize(volume_test_data['pair'])[0]

### GBDT调参